In [1]:
import os

In [3]:
os.chdir("../")

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path
    tokenizer_name: Path

In [6]:
from src.code.constants import *
from src.code.utils.common import read_yaml, create_directories


In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation

        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            tokenizer_name = config.tokenizer_name
        )

        return data_transformation_config

In [8]:
from src.code.logging import LogTool
from transformers import AutoTokenizer
from datasets import load_dataset, load_from_disk

/Users/sujith/anaconda3/envs/quickgist/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2024-01-11 00:18:08,117: INFO: config] PyTorch version 2.1.2 available.


In [13]:
class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config
        self.tokenizer = AutoTokenizer.from_pretrained(config.tokenizer_name)
        
    
    def convert_examples_to_features(self,example_batch):
        input_encodings = self.tokenizer(example_batch['article'] , max_length = 1024, truncation = True)
        
        with self.tokenizer.as_target_tokenizer():
            target_encodings = self.tokenizer(example_batch['highlights'], max_length = 128, truncation = True)
            
        return {
            'input_ids' : input_encodings['input_ids'],
            'attention_mask': input_encodings['attention_mask'],
            'labels': target_encodings['input_ids']
        }
    
    def transform(self):
        try:
            dataset = load_dataset("csv", data_files={'train': f"{self.config.data_path}/train.csv",
                                                        'test': f"{self.config.data_path}/test.csv",
                                                        'validation': f"{self.config.data_path}/validation.csv"})
            
            if len(dataset['train']) == 0:
                raise ValueError("Empty dataset.")
            
            dataset_pt = dataset.map(self.convert_examples_to_features, batched=True)
            dataset_pt.save_to_disk(os.path.join(self.config.root_dir, "dataset"))
        except Exception as e:
            raise ValueError(f"Error during transformation: {e}")

In [14]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.transform()
except Exception as e:
    raise e

[2024-01-11 00:29:02,487: INFO: common] yaml file: configs/config.yaml loaded successfully
[2024-01-11 00:29:02,489: INFO: common] yaml file: param.yaml loaded successfully
[2024-01-11 00:29:02,489: INFO: common] created directory at: artifacts
[2024-01-11 00:29:02,490: INFO: common] created directory at: artifacts/data_transformation


Generating train split: 287113 examples [00:10, 27555.76 examples/s]
Generating test split: 11490 examples [00:00, 26519.20 examples/s]
Generating validation split: 13368 examples [00:00, 25730.65 examples/s]
Map:   0%|          | 0/287113 [00:00<?, ? examples/s]/Users/sujith/anaconda3/envs/quickgist/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:3860: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
Saving the dataset (1/1 shards): 100%|██████████| 13368/13368 [00:00<00:00, 62167.12 examples/s]
